# Cuaderno 4: Problemas de flujo de costo mínimo

$\newcommand{\R}{\mathbb{R}}$
Trataremos en este cuaderno el problema de encontrar un flujo de costo mínimo que satisfaga demandas asociadas a los nodos en una red capacitada, mientras respeta las capacidades de los arcos.
 
## Algoritmo del simplex en redes

Implementaremos el algoritmo del simplex en redes.

Consideremos un grafo dirigido $D=(V, A)$, con capacidades $u \in \R^{A}$ y costos unitarios $c \in \R^{A}$ sobre los arcos, y demandas $b \in \R^{V}$ sobre los nodos. Los datos están dados por medio de las listas `V`, `A` y `b` a indicadas continuación. Los elementos de las lista `A` son tuplas de la forma `(i,j,u_ij,c_ij)`. 

In [3]:
# nodos del grafo
V = range(1, 6)
# arcos, capacidades y costos (i, j, u_ij, c_ij)
A = [(1,2,5,3), (1,4,2,1), (2,3,4,5), (2,5,3,2), (2,4,2,1), (5,4,3,1), (3,5,3,2)]
# demandas de los nodos
b = [-6, 4, -3, 3, 2]

print('V= {}'.format(V))
print('A= {}'.format(A))
print('b= {}'.format(b))


V= range(1, 6)
A= [(1, 2, 5, 3), (1, 4, 2, 1), (2, 3, 4, 5), (2, 5, 3, 2), (2, 4, 2, 1), (5, 4, 3, 1), (3, 5, 3, 2)]
b= [-6, 4, -3, 3, 2]


Podemos representar gráficamente esta instancia usando las bibliotecas `networkx` y `ipycytoscape`:

In [4]:
import networkx as nx
import ipycytoscape
D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['demanda']= str(i) + '\n' + str(b[i-1])
for i,j,u,c in A:
    D.add_edge(i,j)
    D.edges[i,j]['cap_costo'] = u, c
# print(D.edges)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(demanda)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(demanda)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': '#9dbaea', 'font-size': '10px', 'label': 'data(cap_costo)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': '#9dbaea'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'background…

En primer lugar, creamos diccionarios de con arcos entrantes / salientes a cada nodo, así como con los costos y las capacidades de los arcos, y las demandas de los nodos. Como vamos a trabajar con dos tipos de arcos (originales y artificiales) representaremos a cada arco $(i,j)$ por una terna `(i,j,k)` donde el valor de $k$ indica si el arco es original ($k=1$) o artificial ($k=0$).

In [5]:
# extraer las primeras dos componentes de cada arco en la lista AA
AA = [(i, j, 1) for (i,j,uij,cij) in A]
# crear diccionarios de arcos salientes / entrantes
salientes = {i : [(k,j,l) for (k,j,l) in AA if k==i] for i in V}
entrantes = {i : [(k,j,l) for (k,j,l) in AA if j==i] for i in V}

# transformar las capacidades en diccionario indexado por los arcos
uu = {(i,j,1) : uij for (i,j,uij,cij) in A}
# transformar los costos en diccionario indexado por los arcos
cc = {(i,j,1) : cij for (i,j,uij,cij) in A}
# transformar las demandas en diccionario indexado por los nodos
bb ={V[i] : b[i] for i in range(len(V))}

print('AA= {}'.format(AA))
print('sal= {}'.format(salientes))
print('ent= {}'.format(entrantes))
print('uu= {}'.format(uu))
print('cc= {}'.format(cc))
print('bb= {}'.format(bb))


AA= [(1, 2, 1), (1, 4, 1), (2, 3, 1), (2, 5, 1), (2, 4, 1), (5, 4, 1), (3, 5, 1)]
sal= {1: [(1, 2, 1), (1, 4, 1)], 2: [(2, 3, 1), (2, 5, 1), (2, 4, 1)], 3: [(3, 5, 1)], 4: [], 5: [(5, 4, 1)]}
ent= {1: [], 2: [(1, 2, 1)], 3: [(2, 3, 1)], 4: [(1, 4, 1), (2, 4, 1), (5, 4, 1)], 5: [(2, 5, 1), (3, 5, 1)]}
uu= {(1, 2, 1): 5, (1, 4, 1): 2, (2, 3, 1): 4, (2, 5, 1): 3, (2, 4, 1): 2, (5, 4, 1): 3, (3, 5, 1): 3}
cc= {(1, 2, 1): 3, (1, 4, 1): 1, (2, 3, 1): 5, (2, 5, 1): 2, (2, 4, 1): 1, (5, 4, 1): 1, (3, 5, 1): 2}
bb= {1: -6, 2: 4, 3: -3, 4: 3, 5: 2}


Elegimos $r=1$. Construimos el primer árbol de la red con los arcos $(r,i)$, para todo $i \in V \setminus \{r\}$ con $b_i \geq 0$, y los arcos $(i,r)$, para todo $i \in V \setminus \{r\}$ con $b_i < 0$. De ser necesario, agregamos a la red nuevos arcos con costo $M$ y capacidad $\left|b_i\right|$, donde $M$ es una constante suficientemente grande. En nuestro caso, elegimos $M$ igual a la suma de los productos entre los costos y las capacidades sobre todos los arcos de $A$.

In [6]:
# elegimos r
r = 1
# elegimos M como la suma de costos*capacidades de todos los arcos
M = sum([cc[i,j,k]*uu[i,j,k] for (i,j,k) in AA])

# crear lista de arcos faltantes la forma (r,i) con bb[i]>=0
L1 = [(r,i,0) for i in V if i!=r and bb[i]>=0 and (r,i,1) not in salientes[r]]
# agregar a la lista arcos (r,i) con u[r,i]<bb[i]
L1+= [(r,i,0) for i in V if i!=r and bb[i]>=0 and (r,i,1) in salientes[r] and uu[r,i,1]<bb[i]]
# agregar arcos artificiales de la forma (r,i,0) a la red
# la tercera componente sirve para identificar que se trata de un arco artificial
for (r,i,k) in L1:
    AA.append((r,i,k))
    salientes[r].append((r,i,k))
    entrantes[i].append((r,i,k))
    uu[(r,i,k)]= bb[i]
    cc[(r,i,k)]= M

# crear lista de arcos faltantes la forma (i,r) con bb[i]<0
L2 = [(i,r,0) for i in V if i!=r and bb[i]<0 and (i,r,1) not in entrantes[r]]
# agregar a la lista arcos (i,r) con u[i,r]<-bb[i]
L2+= [(i,r,0) for i in V if i!=r and bb[i]<0 and (i,r,1) in entrantes[r] and uu[i,r,1]<-bb[i]]
# agregar arcos faltantes de la forma (i,r) a la red
# la tercera componente sirve para identificar que se trata de un arco artificial
for (i,r,k) in L2:
    AA.append((i,r,k))
    entrantes[r].append((i,r,k))
    salientes[i].append((i,r,k))
    uu[(i,r,k)]= -bb[i]
    cc[(i,r,k)]= M
    
# lista con los arcos del arbol inicial
# son todos los arcos artificiales añadidos...
T = L1 + L2
# ... más los arcos originales que sirvan para enviar el flujo inicial
T+= [(r,i,1) for i in V if i!=r and bb[i]>=0 and (r,i,1) in salientes[r] and uu[r,i,1]>=bb[i]]
T+= [(i,r,1) for i in V if i!=r and bb[i]<0 and (i,r,1) in entrantes[r] and uu[i,r,1]>=-bb[i]]

print('AA= {}'.format(AA))
print('sal= {}'.format(salientes))
print('ent= {}'.format(entrantes))
print('uu= {}'.format(uu))
print('cc= {}'.format(cc))
print('bb= {}'.format(bb))
print('T= {}'.format(T))
print('M= {}'.format(M))


AA= [(1, 2, 1), (1, 4, 1), (2, 3, 1), (2, 5, 1), (2, 4, 1), (5, 4, 1), (3, 5, 1), (1, 5, 0), (1, 4, 0), (3, 1, 0)]
sal= {1: [(1, 2, 1), (1, 4, 1), (1, 5, 0), (1, 4, 0)], 2: [(2, 3, 1), (2, 5, 1), (2, 4, 1)], 3: [(3, 5, 1), (3, 1, 0)], 4: [], 5: [(5, 4, 1)]}
ent= {1: [(3, 1, 0)], 2: [(1, 2, 1)], 3: [(2, 3, 1)], 4: [(1, 4, 1), (2, 4, 1), (5, 4, 1), (1, 4, 0)], 5: [(2, 5, 1), (3, 5, 1), (1, 5, 0)]}
uu= {(1, 2, 1): 5, (1, 4, 1): 2, (2, 3, 1): 4, (2, 5, 1): 3, (2, 4, 1): 2, (5, 4, 1): 3, (3, 5, 1): 3, (1, 5, 0): 2, (1, 4, 0): 3, (3, 1, 0): 3}
cc= {(1, 2, 1): 3, (1, 4, 1): 1, (2, 3, 1): 5, (2, 5, 1): 2, (2, 4, 1): 1, (5, 4, 1): 1, (3, 5, 1): 2, (1, 5, 0): 54, (1, 4, 0): 54, (3, 1, 0): 54}
bb= {1: -6, 2: 4, 3: -3, 4: 3, 5: 2}
T= [(1, 5, 0), (1, 4, 0), (3, 1, 0), (1, 2, 1)]
M= 54


Nuevamente, empleamos `networkx` y `ipycytoscape` para representar al grafo con los arcos artificiales (en rojo) y el árbol inicial (en rojo y azul oscuro):

In [7]:
import networkx as nx
import ipycytoscape
D = nx.MultiDiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['demanda']= str(i) + '\n' + str(bb[i])
for i,j,k in AA:
    D.add_edge(i,j,key=(i,j,k))
    D.edges[i,j,(i,j,k)]['cap_costo'] = uu[i,j,k], cc[i,j,k]
    if k==0:
        D.edges[i,j,(i,j,k)]['color'] = '#ff0040'
    elif (i,j,k) in T:
        D.edges[i,j,(i,j,k)]['color'] = '#0404b4'
    else:
        D.edges[i,j,(i,j,k)]['color'] = '#bdbdbd'
# print(D.edges)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(demanda)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(demanda)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(cap_costo)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'background…

Construimos un flujo factible de árbol inicial de la forma $x_{ri}=b_i$ para todo $(r,i) \in T$, $x_{ir}=-b_i$ para todo $(i,r) \in T$ y $x_{ij}=0$ si $(i,j) \not\in T$. Para poder utilizar el operador `+` de concatenación, construimos primero una lista `Lx` de elementos $(i,j,x_{ij})$ y transformamos luego esta lista en un diccionario `x` con los valores del flujo indexados por los arcos. La partición asociada a esta solución es $(L, U, T)$, con $L:= A \setminus T$ y $U:= \emptyset$.

In [8]:
# construir partición inicial
# todos los arcos fuera de T están en L
L = [(i,j,k) for (i,j,k) in AA if (i,j,k) not in T]
U = []

# construir flujo inicial como lista
# flujo sobre arcos de T que salen de r
x = {(i,j,k) : bb[j] for (i,j,k) in T if i==r}
# flujo sobre arcos de T que llegan a r
x.update({(i,j,k) : -bb[i] for (i,j,k) in T if j==r})
# flujo sobre arcos en L
x.update({(i,j,k) : 0 for (i,j,k) in L})

print('x= {}'.format(x))
print('L= {}'.format(L))
print('U= {}'.format(U))
print('T= {}'.format(T))



x= {(1, 5, 0): 2, (1, 4, 0): 3, (1, 2, 1): 4, (3, 1, 0): 3, (1, 4, 1): 0, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 0, (5, 4, 1): 0, (3, 5, 1): 0}
L= [(1, 4, 1), (2, 3, 1), (2, 5, 1), (2, 4, 1), (5, 4, 1), (3, 5, 1)]
U= []
T= [(1, 5, 0), (1, 4, 0), (3, 1, 0), (1, 2, 1)]


Empleamos un algoritmo de marcaje BFS para calcular los valores de $y_i$, para $i \in V$, como los costos de los senderos desde $r$ en el árbol $T$. Simultáneamente, construimos un diccionario `p` que almacena, para cada nodo $i \in V$, el nodo antecesor en el sendero único desde $r$ hasta $i$. 

In [9]:
# construimos diccionarios de arcos entrantes y salientes en T
Tsal = {i : [(k,j,l) for (k,j,l) in T if k==i] for i in V}
Tent = {i : [(k,j,l) for (k,j,l) in T if j==i] for i in V}
# inicializamos y_r = 0 y y_i=None para los demás nodos
y = {i : None for i in V}
y[r] = 0
# inicializamos p_i=None para todos los nodos
p = {i : None for i in V}
print('y = {}'.format(y))
print('p = {}'.format(p))
# colocar r en la cola de nodos por procesar
Q = [r]
# continuar mientras Q no esté vacía
while Q!=[]:
    # retirar primer elemento de Q
    i = Q.pop(0)
    print('i = {}'.format(i))
    # examinar arcos salientes de i
    for (i,j,k) in Tsal[i]:
        # marcar el otro extremo, si no está marcado, y ponerlo en Q
        if y[j] == None:
            y[j] = y[i] + cc[i,j,k]
            p[j] = i
            Q.append(j)
    # examinar arcos entrantes a i
    for (j,i,k) in Tent[i]:
        # marcar el otro extremo, si no está marcado, y ponerlo en Q
        if y[j] == None:
            y[j] = y[i] - cc[j,i,k]
            p[j]= i
            Q.append(j)
    print('Q = {}'.format(Q))
    print('y = {}'.format(y))
    print('p = {}'.format(p))
    
    

y = {1: 0, 2: None, 3: None, 4: None, 5: None}
p = {1: None, 2: None, 3: None, 4: None, 5: None}
i = 1
Q = [5, 4, 2, 3]
y = {1: 0, 2: 3, 3: -54, 4: 54, 5: 54}
p = {1: None, 2: 1, 3: 1, 4: 1, 5: 1}
i = 5
Q = [4, 2, 3]
y = {1: 0, 2: 3, 3: -54, 4: 54, 5: 54}
p = {1: None, 2: 1, 3: 1, 4: 1, 5: 1}
i = 4
Q = [2, 3]
y = {1: 0, 2: 3, 3: -54, 4: 54, 5: 54}
p = {1: None, 2: 1, 3: 1, 4: 1, 5: 1}
i = 2
Q = [3]
y = {1: 0, 2: 3, 3: -54, 4: 54, 5: 54}
p = {1: None, 2: 1, 3: 1, 4: 1, 5: 1}
i = 3
Q = []
y = {1: 0, 2: 3, 3: -54, 4: 54, 5: 54}
p = {1: None, 2: 1, 3: 1, 4: 1, 5: 1}


In [12]:
D = nx.MultiDiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['demanda']= str(i) + '\n' + '{}/{}/{}'.format(bb[i],y[i],p[i])
for i,j,k in AA:
    D.add_edge(i,j,key=(i,j,k))
    D.edges[i,j,(i,j,k)]['cap_costo'] = x[i,j,k], uu[i,j,k], cc[i,j,k]
    if (i,j,k) in T:
        if k==0:
            D.edges[i,j,(i,j,k)]['color'] = '#ff0040'
        else:
            D.edges[i,j,(i,j,k)]['color'] = '#0404b4'
    else:
        D.edges[i,j,(i,j,k)]['color'] = '#bdbdbd'
# print(D.edges)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '6px', 'color':'white', 'label': 'data(demanda)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(demanda)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(cap_costo)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'background…

Para poder determinar un ciclo $x$-aumentante, necesitaremos calcular la capacidad del sendero único desde el nodo `i` hasta el nodo `j` sobre el árbol. Implementamos para ello la función `reconstruir_sendero`. Esta función retorna la capacidad del sendero y una lista con sus arcos. 

In [13]:
def reconstruir_sendero(i, j, p, uu, x, T):
    # lista de nodos del sendero de i hasta r
    Pi, i2 = [i], i
    while p[i2]!= None:
        # mientras i2!=r, agregar el predecesor de i2 en Pi y actualizar i2
        Pi.append(p[i2])
        i2 = p[i2]
    # print("Pi : {}".format(Pi))
    # recorremos los nodos del sendero j hasta r, parando antes del primer nodo común con Pi
    Pj, j2 = [], j
    # si j no pertenece a Pi, reconstruir el sendero desde j hasta r
    if j2 not in Pi:
        while p[j2]!= None:
            # terminar al llegar a un nodo de Pi
            if j2 in Pi:
                break
            # insertar al inicio, para invertir el orden
            Pj.insert(0, j2)
            j2 = p[j2]
    # encontrar índice del primer nodo común de Pi y Pj
    ind = Pi.index(j2)
    # nodos del sendero de i a j:
    Pi= Pi[0:ind+1] + Pj
    # en este punto, Pi contiene los nodos del sendero de i hasta j
    # print("Pi : {}".format(Pi))
    
    # Construir lista P con arcos del sendero de i a j:
    P = []
    adelante = {} # marca si los arcos son "hacia adelante"
    for k in range(len(Pi)-1):
        if (Pi[k], Pi[k+1], 1) in T:
            P.append((Pi[k], Pi[k+1], 1))
            adelante[Pi[k], Pi[k+1], 1]= True
        elif (Pi[k], Pi[k+1], 0) in T:
            P.append((Pi[k], Pi[k+1], 0))
            adelante[Pi[k], Pi[k+1], 0]= True
        elif (Pi[k+1], Pi[k], 1) in T:
            P.append((Pi[k+1], Pi[k], 1))
            adelante[Pi[k+1], Pi[k], 1]= False
        else:
            P.append((Pi[k+1], Pi[k], 0))
            adelante[Pi[k+1], Pi[k], 0]= False
    # capacidad del sendero
    eps = min([uu[i,j,k] - x[i,j,k] if adelante[i,j,k] 
               else x[i,j,k] for (i,j,k) in P])
    return eps, P

eps, P = reconstruir_sendero(2, 4, p, uu, x, T)
print("Sendero de 2 a 4: {}".format(P))
print("Capacidad: {}".format(eps))

eps, P = reconstruir_sendero(4, 2, p, uu, x, T)
print("Sendero de 4 a 2: {}".format(P))
print("Capacidad: {}".format(eps))

eps, P = reconstruir_sendero(3, 4, p, uu, x, T)
print("Sendero de 3 a 4: {}".format(P))
print("Capacidad: {}".format(eps))

eps, P = reconstruir_sendero(2, 1, p, uu, x, T)
print("Sendero de 2 a 1: {}".format(P))
print("Capacidad: {}".format(eps))

eps, P = reconstruir_sendero(5, 3, p, uu, x, T)
print("Sendero de 5 a 3: {}".format(P))
print("Capacidad: {}".format(eps))


Sendero de 2 a 4: [(1, 2, 1), (1, 4, 0)]
Capacidad: 0
Sendero de 4 a 2: [(1, 4, 0), (1, 2, 1)]
Capacidad: 1
Sendero de 3 a 4: [(3, 1, 0), (1, 4, 0)]
Capacidad: 0
Sendero de 2 a 1: [(1, 2, 1)]
Capacidad: 4
Sendero de 5 a 3: [(1, 5, 0), (3, 1, 0)]
Capacidad: 2


En cada iteración del algoritmo del simplex en redes, debemos determinar, si existe, una arista de `L` con costo reducido negativo, o una arista de `U` con costo reducido positivo. 

In [14]:
# aristas de L con costo reducido negativo
Lneg = [(i,j,k) for (i,j,k) in L if cc[i,j,k]+y[i]-y[j]<0]
# aristas de U con costo reducido positivo
Upos = [(i,j,k) for (i,j,k) in U if cc[i,j,k]+y[i]-y[j]>0]

print('Lneg = {}'.format(Lneg))
print('Upos = {}'.format(Upos))


Lneg = [(1, 4, 1), (2, 5, 1), (2, 4, 1), (3, 5, 1)]
Upos = []


Empleando la función `reconstruir_sendero` es ahora posible construir el ciclo $x$-aumentante asociado a un arco de `Lneg` o de `Upos`.

In [15]:
# construir el ciclo asociado al primer arco de Lneg
(i, j, k) = Lneg.pop(0)
eps, P = reconstruir_sendero(j, i, p, uu, x, T)
# capacidad disponible en (i,j) es uu[i,j], porque x[i,j]==0
eps = min(eps, uu[i,j,k])
P.insert(0, (i,j,k))

print("Ciclo asociado a {}: {}".format((i,j,k), P))
print("Capacidad: {}".format(eps))

Ciclo asociado a (1, 4, 1): [(1, 4, 1), (1, 4, 0)]
Capacidad: 2


Con esta idea, implementamos la función `reconstruir_ciclo`, que retorna la lista de arcos del ciclo asociado a un arco de `L` o de `U`, conjuntamente con su capacidad. El parámetro `enL` toma el valor de `True` si el arco `(i,j)` pertenece a `L`, y `False` si se trata de un arco de `U`.

In [16]:
def reconstruir_ciclo(i, j, k, p, uu, x, T, enL):
    # para arcos en L:
    if enL:
        # reconstruir el sendero de j a i
        eps, P = reconstruir_sendero(j, i, p, uu, x, T)
        # agregar arco (i,j) y de ser necesario actualizar eps
        eps = min(eps, uu[i,j,k])
        P.insert(0, (i,j,k))
    else:
        # reconstruir el sendero de i a j
        eps, P = reconstruir_sendero(i, j, p, uu, x, T)
        # agregar arco (i,j) y de ser necesario actualizar eps
        eps = min(eps, x[i,j,k])
        P.insert(0, (i,j,k))
    return eps, P    
    
(i, j, k) = Lneg.pop(0)
eps, P = reconstruir_ciclo(i, j, k, p, uu, x, T, True)
print("Ciclo asociado a {}: {}".format((i,j,k), P))
print("Capacidad: {}".format(eps))   


Ciclo asociado a (2, 5, 1): [(2, 5, 1), (1, 5, 0), (1, 2, 1)]
Capacidad: 1


Finalmente, implementemos una función `transformar_flujo` que transforma el flujo utilizando un ciclo $x$-aumentante. La función retorna una lista `U2` con los arcos hacia adelante del ciclo que se saturaron durante la transformación, y una lista `L2` con los arcos en reversa que se vaciaron durante la transformación.

In [17]:
def transformar_flujo(C, eps, i, x, uu):
    # lista con los arcos que pueden salir del árbol
    salir = []
    for (j,k,l) in C:
        # si es arco hacia adelante:
        if i==j:
            # aumentar flujo
            x[j,k,l]+= eps
            # si el arco se satura, agregarlo a U2
            if x[j,k,l]==uu[j,k,l]:
                salir.append((j,k,l))
            # actualizar i
            i = k
        # si es arco en reversa:
        else:
            # disminuir flujo
            x[j,k,l]-= eps
            # si el arco se vacía, agregarlo a L2
            if x[j,k,l]==0:
                salir.append((j,k,l))
            # actualizar i
            i = j
    return salir

print("Flujo inicial: {}".format(x))
(i, j, k) = Lneg.pop(0)
eps, C = reconstruir_ciclo(i, j, k, p, uu, x, T, True)
print("Ciclo asociado a {}: {}".format((i,j,k), C))
print("Capacidad: {}".format(eps))   
salir = transformar_flujo(C, eps, i, x, uu)
print("Nuevo flujo: {}".format(x))
print("Candidatos a salir de T = {}".format(salir))
            


Flujo inicial: {(1, 5, 0): 2, (1, 4, 0): 3, (1, 2, 1): 4, (3, 1, 0): 3, (1, 4, 1): 0, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 0, (5, 4, 1): 0, (3, 5, 1): 0}
Ciclo asociado a (2, 4, 1): [(2, 4, 1), (1, 4, 0), (1, 2, 1)]
Capacidad: 1
Nuevo flujo: {(1, 5, 0): 2, (1, 4, 0): 2, (1, 2, 1): 5, (3, 1, 0): 3, (1, 4, 1): 0, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 1, (5, 4, 1): 0, (3, 5, 1): 0}
Candidatos a salir de T = [(1, 2, 1)]


Luego de actualizar el flujo, es necesario actualizar la partición. El primer arco de la lista `salir` que sea distinto de `(i,j,k)` debe salir de `T` e ingresar a `L` o `U`, (según corresponda), y el arco `(i,j,k)` debe ingresar a `T` y salir de `L` o `U`, según corresponda. Si el único arco en la lista `salir` es `(i,j,k)`, entonces este arco debe pasar de `L` a `U`, o de `U` a `L`, según corresponda. En este último caso, el árbol `T` permanece inalterado.

Implementamos la función `actualizar_particion` para ejecutar esta tarea. El parámetro `enL` vale `True` si el arco `(i,j,k)` pertenece a `L` y `False` si pertenece a `U`. 

In [18]:
def actualizar_particion(T, L, U, salir, i, j, k, x, enL):
    # remover (i,j,k) de L o U, según corresponda
    if enL:
        L.remove((i,j,k))
    else:
        U.remove((i,j,k))
    # si (i,j,k) es el único arco en la lista salir, agregarlo a L o U, según corresponda        
    if len(salir)==1 and (i,j,k) in salir:
        if enL:
            U.append((i,j,k))
        else:
            L.append((i,j,k))
    else:
        # caso contrario, mover (i,j,k) hacia T
        T.append((i,j,k))
        # determinar arco saliente    
        (h,l,k2) = [(h,l,k2) for (h,l,k2) in salir if (h,l,k2)!=(i,j,k)].pop(0)
        # mover (h,l,k2) desde T a L o U, según corresponda
        T.remove((h,l,k2))
        if x[h,l,k2]==0:
            L.append((h,l,k2))
        else:
            U.append((h,l,k2))


print("(i,j,k) = {}".format((i,j,k)))
print("salir = {}".format(salir))
print("T = {}".format(T))
print("L = {}".format(L))
print("U = {}".format(U))

actualizar_particion(T, L, U, salir, i, j, k, x, True)
print("---")

print("T = {}".format(T))
print("L = {}".format(L))
print("U = {}".format(U))


(i,j,k) = (2, 4, 1)
salir = [(1, 2, 1)]
T = [(1, 5, 0), (1, 4, 0), (3, 1, 0), (1, 2, 1)]
L = [(1, 4, 1), (2, 3, 1), (2, 5, 1), (2, 4, 1), (5, 4, 1), (3, 5, 1)]
U = []
---
T = [(1, 5, 0), (1, 4, 0), (3, 1, 0), (2, 4, 1)]
L = [(1, 4, 1), (2, 3, 1), (2, 5, 1), (5, 4, 1), (3, 5, 1)]
U = [(1, 2, 1)]


Para terminar la primera iteración del lazo principal del algoritmo del simplex, debemos actualizar los valores de `y` y `p`. Implementamos para ello la función `actualizar_y` basada en la búsqueda BFS presentada arriba:

In [19]:
def actualizar_y(T, V, r, cc):
    # construimos diccionarios de arcos entrantes y salientes en T
    Tsal = {i : [(k,j,l) for (k,j,l) in T if k==i] for i in V}
    Tent = {i : [(k,j,l) for (k,j,l) in T if j==i] for i in V}
    # inicializamos y_r = 0 y y_i=None para los demás nodos
    y = {i : None for i in V}
    y[r] = 0
    # inicializamos p_i=None para todos los nodos
    p = {i : None for i in V}
    # colocar r en la cola de nodos por procesar
    Q = [r]
    # continuar mientras Q no esté vacía
    while Q!=[]:
        # retirar primer elemento de Q
        i = Q.pop(0)
        # examinar arcos salientes de i
        for (i,j,k) in Tsal[i]:
            # marcar el otro extremo, si no está marcado, y ponerlo en Q
            if y[j] == None:
                y[j] = y[i] + cc[i,j,k]
                p[j] = i
                Q.append(j)
        # examinar arcos entrantes a i
        for (j,i,k) in Tent[i]:
            # marcar el otro extremo, si no está marcado, y ponerlo en Q
            if y[j] == None:
                y[j] = y[i] - cc[j,i,k]
                p[j]= i
                Q.append(j)
    return y, p


print("T= {}".format(T))
y, p = actualizar_y(T, V, r, cc)
print("y= {}".format(y))
print("p= {}".format(p))
                



T= [(1, 5, 0), (1, 4, 0), (3, 1, 0), (2, 4, 1)]
y= {1: 0, 2: 53, 3: -54, 4: 54, 5: 54}
p= {1: None, 2: 4, 3: 1, 4: 1, 5: 1}


Con esto podemos implementar el lazo principal del algoritmo del simplex en redes:

In [20]:
# actualizar las listas con arcos candidatos Lneg y Upos
# aristas de L con costo reducido negativo
Lneg = [(i,j,k) for (i,j,k) in L if cc[i,j,k]+y[i]-y[j]<0]
# aristas de U con costo reducido positivo
Upos = [(i,j,k) for (i,j,k) in U if cc[i,j,k]+y[i]-y[j]>0]

print('Lneg = {}'.format(Lneg))
print('Upos = {}'.format(Upos))

# lazo principal del algoritmo
# mientras Lneg y Upos no sean ambas vacías
while Lneg!=[] or Upos!=[]:
    # seleccionar arco candidato a ingresar al árbol
    if Lneg!=[]:
        (i,j,k) = Lneg.pop(0)
        enL = True
    else:
        (i,j,k) = Upos.pop(0)
        enL = False
    print("(i,j,k): {}".format((i,j,k)))
    # reconstruir ciclo asociado a (i,j)
    eps, C = reconstruir_ciclo(i, j, k, p, uu, x, T, enL)
    print("C: {}".format(C))
    print("eps= {}".format(eps))
    # transformar el flujo usando C
    if enL:
        salir = transformar_flujo(C, eps, i, x, uu)
    else:
        salir = transformar_flujo(C, eps, j, x, uu)
    print("x= {}".format(x))
    # actualizar particion
    # print("T: {}".format(T))
    # print("L: {}".format(L))
    # print("U: {}".format(U))
    actualizar_particion(T, L, U, salir, i, j, k, x, enL)
    print("T: {}".format(T))
    print("L: {}".format(L))
    print("U: {}".format(U))
    # actualizar y y p
    y, p = actualizar_y(T, V, r,cc)
    print("y= {}".format(y))
    print("p= {}".format(p))
    # actualizar las listas con arcos candidatos Lneg y Upos
    # aristas de L con costo reducido negativo
    Lneg = [(i,j,k) for (i,j,k) in L if cc[i,j,k]+y[i]-y[j]<0]
    print("Lneg: {}".format(Lneg))
    # aristas de U con costo reducido positivo
    Upos = [(i,j,k) for (i,j,k) in U if cc[i,j,k]+y[i]-y[j]>0]
    print("Upos: {}".format(Upos))


Lneg = [(1, 4, 1), (3, 5, 1)]
Upos = []
(i,j,k): (1, 4, 1)
C: [(1, 4, 1), (1, 4, 0)]
eps= 2
x= {(1, 5, 0): 2, (1, 4, 0): 0, (1, 2, 1): 5, (3, 1, 0): 3, (1, 4, 1): 2, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 1, (5, 4, 1): 0, (3, 5, 1): 0}
T: [(1, 5, 0), (3, 1, 0), (2, 4, 1), (1, 4, 1)]
L: [(2, 3, 1), (2, 5, 1), (5, 4, 1), (3, 5, 1), (1, 4, 0)]
U: [(1, 2, 1)]
y= {1: 0, 2: 0, 3: -54, 4: 1, 5: 54}
p= {1: None, 2: 4, 3: 1, 4: 1, 5: 1}
Lneg: [(2, 5, 1), (3, 5, 1)]
Upos: [(1, 2, 1)]
(i,j,k): (2, 5, 1)
C: [(2, 5, 1), (1, 5, 0), (1, 4, 1), (2, 4, 1)]
eps= 0
x= {(1, 5, 0): 2, (1, 4, 0): 0, (1, 2, 1): 5, (3, 1, 0): 3, (1, 4, 1): 2, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 1, (5, 4, 1): 0, (3, 5, 1): 0}
T: [(1, 5, 0), (3, 1, 0), (2, 4, 1), (2, 5, 1)]
L: [(2, 3, 1), (5, 4, 1), (3, 5, 1), (1, 4, 0)]
U: [(1, 2, 1), (1, 4, 1)]
y= {1: 0, 2: 52, 3: -54, 4: 53, 5: 54}
p= {1: None, 2: 5, 3: 1, 4: 2, 5: 1}
Lneg: [(3, 5, 1)]
Upos: []
(i,j,k): (3, 5, 1)
C: [(3, 5, 1), (1, 5, 0), (3, 1, 0)]
eps= 2
x= {(1, 5, 0): 

Reunimos finalmente todo el código anterior en una función para calcular el flujo de costo mínimo de una red capacitada con demandas empleando el algoritmo del simplex en redes:

In [21]:
def simplex_redes(V, A, b):
    # extraer las primeras dos componentes de cada arco en la lista AA
    AA = [(i, j, 1) for (i,j,uij,cij) in A]
    # crear diccionarios de arcos salientes / entrantes
    salientes = {i : [(k,j,l) for (k,j,l) in AA if k==i] for i in V}
    entrantes = {i : [(k,j,l) for (k,j,l) in AA if j==i] for i in V}

    # transformar las capacidades en diccionario indexado por los arcos
    uu = {(i,j,1) : uij for (i,j,uij,cij) in A}
    # transformar los costos en diccionario indexado por los arcos
    cc = {(i,j,1) : cij for (i,j,uij,cij) in A}
    # transformar las demandas en diccionario indexado por los nodos
    bb ={V[i] : b[i] for i in range(len(V))}

    # elegir nodo r
    r = 1

    # Crear árbol inicial
    # elegir M como la suma de costos*capacidades de todos los arcos
    M = sum([cc[i,j,k]*uu[i,j,k] for (i,j,k) in AA])
    # crear lista de arcos faltantes la forma (r,i) con bb[i]>=0
    L1 = [(r,i,0) for i in V if i!=r and bb[i]>=0 and (r,i,1) not in salientes[r]]
    # agregar a la lista arcos (r,i) con u[r,i]<bb[i]
    L1+= [(r,i,0) for i in V if i!=r and bb[i]>=0 and (r,i,1) in salientes[r] and uu[r,i,1]<bb[i]]
    # agregar arcos artificiales de la forma (r,i,0) a la red
    # la tercera componente sirve para identificar que se trata de un arco artificial
    for (r,i,k) in L1:
        AA.append((r,i,k))
        salientes[r].append((r,i,k))
        entrantes[i].append((r,i,k))
        uu[(r,i,k)]= bb[i]
        cc[(r,i,k)]= M
    # crear lista de arcos faltantes la forma (i,r) con bb[i]<0
    L2 = [(i,r,0) for i in V if i!=r and bb[i]<0 and (i,r,1) not in entrantes[r]]
    # agregar a la lista arcos (i,r) con u[i,r]<-bb[i]
    L2+= [(i,r,0) for i in V if i!=r and bb[i]<0 and (i,r,1) in entrantes[r] and uu[i,r,1]<-bb[i]]
    # agregar arcos faltantes de la forma (i,r) a la red
    # la tercera componente sirve para identificar que se trata de un arco artificial
    for (i,r,k) in L2:
        AA.append((i,r,k))
        entrantes[r].append((i,r,k))
        salientes[i].append((i,r,k))
        uu[(i,r,k)]= -bb[i]
        cc[(i,r,k)]= M
    # lista con los arcos del arbol inicial
    # son todos los arcos artificiales añadidos...
    T = L1 + L2
    # ... más los arcos originales que sirvan para enviar el flujo inicial
    T+= [(r,i,1) for i in V if i!=r and bb[i]>=0 and (r,i,1) in salientes[r] and uu[r,i,1]>=bb[i]]
    T+= [(i,r,1) for i in V if i!=r and bb[i]<0 and (i,r,1) in entrantes[r] and uu[i,r,1]>=-bb[i]]

    # construir flujo inicial como lista
    Lx = [(r,i,k,bb[i]) for (j,i,k) in T if j==r] + [(i,r,k,-bb[i]) for (i,j,k) in T if j==r]
    Lx+= [(i,j,k,0) for (i,j,k) in AA if (i,j,k) not in T]
    # transformar flujo inicial en diccionario
    x = {(i,j,k) : xij for (i,j,k,xij) in Lx}

    # construir partición inicial
    L = [(i,j,k) for (i,j,k) in AA if (i,j,k) not in T]
    U = []

    # inicializar y y p
    y, p = actualizar_y(T, V, r, cc)
    # actualizar las listas con arcos candidatos Lneg y Upos
    # aristas de L con costo reducido negativo
    Lneg = [(i,j,k) for (i,j,k) in L if cc[i,j,k]+y[i]-y[j]<0]
    # aristas de U con costo reducido positivo
    Upos = [(i,j,k) for (i,j,k) in U if cc[i,j,k]+y[i]-y[j]>0]
    
    # lazo principal del algoritmo
    # mientras Lneg y Upos no sean ambas vacías
    while Lneg!=[] or Upos!=[]:
        # seleccionar arco candidato a ingresar al árbol
        if Lneg!=[]:
            (i,j,k) = Lneg.pop(0)
            enL = True
        else:
            (i,j,k) = Upos.pop(0)
            enL = False
        # reconstruir ciclo asociado a (i,j)
        eps, C = reconstruir_ciclo(i, j, k, p, uu, x, T, enL)
        # transformar el flujo usando C
        if enL:
            salir = transformar_flujo(C, eps, i, x, uu)
        else:
            salir = transformar_flujo(C, eps, j, x, uu)
        # actualizar particion
        actualizar_particion(T, L, U, salir, i, j, k, x, enL)
        # actualizar y y p
        y, p = actualizar_y(T, V, r, cc)
        # actualizar las listas con arcos candidatos Lneg y Upos
        Lneg = [(i,j,k) for (i,j,k) in L if cc[i,j,k]+y[i]-y[j]<0]
        Upos = [(i,j,k) for (i,j,k) in U if cc[i,j,k]+y[i]-y[j]>0]
    return x

# nodos del grafo
V = range(1, 6)
# arcos, capacidades y costos (i, j, u_ij, c_ij)
A = [(1,2,4,3), (1,4,2,1), (2,3,4,5), (2,5,3,2), (2,4,2,1), (5,4,3,1), (3,5,3,2)]
# demandas de los nodos
b = [-6, 4, -3, 3, 2]

x = simplex_redes(V, A, b)
print("x= {}".format(x))

x= {(1, 5, 0): 0, (1, 4, 0): 0, (1, 2, 1): 4, (3, 1, 0): 0, (1, 4, 1): 2, (2, 3, 1): 0, (2, 5, 1): 0, (2, 4, 1): 0, (5, 4, 1): 1, (3, 5, 1): 3}


Podemos graficar la solución encontrada por el algoritmo empleando las bibliotecas `networkx` y `ipycytoscape`:

In [22]:
import networkx as nx
import ipycytoscape
D = nx.MultiDiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['label']= str(i) + '\n' + str(bb[i]) + '/' + str(y[i])
for i,j,k in AA:
    if x[i,j,k]>0 or k==1:
        D.add_edge(i,j,key=(i,j,k))
        D.edges[i,j,(i,j,k)]['label'] = x[i,j,k], uu[i,j,k], cc[i,j,k]
        if k==0:
            # arcos artificiales en rojo
            D.edges[i,j,(i,j,k)]['color'] = '#ff0040'
        elif (i,j,k) in T:
            # arcos azules en T
            D.edges[i,j,(i,j,k)]['color'] = '#0404b4'
        elif (i,j,k) in L:
            # arcos grises en L
            D.edges[i,j,(i,j,k)]['color'] = '#bdbdbd'
        else:
            # arcos celestes en U
            D.edges[i,j,(i,j,k)]['color'] = '#81bef7'
# print(D.edges)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(label)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(label)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(label)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'background…